<a href="https://colab.research.google.com/github/naubass/AI-Agent-HR-Assistant-Helpdesk-Internal/blob/main/Sentiment_Analyst_Review_Customers_with_IBM_Granite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle pandas scikit-learn nltk --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 754.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.9 MB/s eta 0:00:00


In [2]:
import os
from google.colab import files

print("Please upload your kaggle.json file")
uploaded = files.upload()

# pindahkan file
if 'kaggle.json' in uploaded:
  # buat directory baru
  if not os.path.exists('/root/.kaggle'):
    os.makedirs('/root/.kaggle')

  # pindahkan file dan atur izin
  os.rename('kaggle.json', '/root/.kaggle/kaggle.json')
  os.chmod('/root/.kaggle/kaggle.json', 600)
  print("\nFile kaggle.json berhasil diunggah dan dikonfigurasi!")
else:
    print("\nFile kaggle.json tidak ditemukan. Silakan coba lagi.")

Please upload your kaggle.json file


Saving kaggle.json to kaggle.json

File kaggle.json berhasil diunggah dan dikonfigurasi!


In [3]:
# Unduh dataset dari Kaggle
!kaggle datasets download -d nicapotato/womens-ecommerce-clothing-reviews

# Unzip dataset (biasanya file zip akan memiliki nama yang sama)
!unzip -o womens-ecommerce-clothing-reviews.zip

Dataset URL: https://www.kaggle.com/datasets/nicapotato/womens-ecommerce-clothing-reviews
License(s): CC0-1.0
  0% 0.00/2.79M [00:00<?, ?B/s]
100% 2.79M/2.79M [00:00<00:00, 407MB/s]
Archive:  womens-ecommerce-clothing-reviews.zip
  inflating: Womens Clothing E-Commerce Reviews.csv  


In [5]:
import pandas as pd
from nltk.corpus import stopwords
import nltk
import re

nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

# load dataset
df = pd.read_csv('Womens Clothing E-Commerce Reviews.csv')
df = df[['Review Text', 'Recommended IND', 'Rating']]

# delete review is empty
df = df.dropna(subset=['Review Text'])

# function clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# apply clean text
df['Cleaned Review'] = df['Review Text'].apply(clean_text)

print("Data berhasil dimuat dan dibersihkan.")
print(f"Jumlah total ulasan: {len(df)}")
df.head()


Data berhasil dimuat dan dibersihkan.
Jumlah total ulasan: 22641


,Review Text,Recommended IND,Rating,Cleaned Review
0,Absolutely wonderful - silky and sexy and comf...,1,4,absolutely wonderful silky sexy comfortable
1,Love this dress! it's sooo pretty. i happene...,1,5,love dress sooo pretty happened find store im ...
2,I had such high hopes for this dress and reall...,0,3,high hopes dress really wanted work initially ...
3,"I love, love, love this jumpsuit. it's fun, fl...",1,5,love love love jumpsuit fun flirty fabulous ev...
4,This shirt is very flattering to all due to th...,1,5,shirt flattering due adjustable front tie perf...


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

X = df['Cleaned Review']
y = df['Recommended IND']

# split text
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=1000))
])

# Latih model
print("Melatih model klasifikasi sentimen...")
model_pipeline.fit(X_train, y_train)
print("Model selesai dilatih!")

# eval model
y_pred = model_pipeline.predict(X_test)
print("\n--- Hasil Evaluasi Model ---")
print(f"Akurasi: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred, target_names=['Negatif (0)', 'Positif (1)']))

classification_model = model_pipeline



Melatih model klasifikasi sentimen...
Model selesai dilatih!

--- Hasil Evaluasi Model ---
Akurasi: 0.8905
              precision    recall  f1-score   support

 Negatif (0)       0.80      0.52      0.63       812
 Positif (1)       0.90      0.97      0.94      3717

    accuracy                           0.89      4529
   macro avg       0.85      0.75      0.78      4529
weighted avg       0.88      0.89      0.88      4529



In [7]:
!pip install replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 2.6 MB/s eta 0:00:00


In [16]:
import replicate
import os
from google.colab import userdata

# replicate API
REPLICATE_API_TOKEN = userdata.get('REPLICATE')

if REPLICATE_API_TOKEN.startswith("r8_") and len(REPLICATE_API_TOKEN) > 20:
    os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN
    print("Kredensial Replicate telah diatur.")
else:
    print("PERINGATAN: Harap isi REPLICATE_API_TOKEN dengan token asli dari https://replicate.com/account")

def summarize_review(review_text):
    if "REPLICATE_API_TOKEN" not in os.environ:
        return "[Model Replicate belum dikonfigurasi. Periksa API Token Anda.]"

    model_identifier = "ibm-granite/granite-3.3-8b-instruct" # Model pilihan Anda

    # prompt instructin
    prompt_instruction = f"""<|system|>
      You are a helpful assistant. Your task is to summarize a customer review in one concise sentence.
      <|user|>
      Summarize this review:
      {review_text}
      <|assistant|>
    """

    input_data = {
        "prompt": prompt_instruction,
        "max_tokens": 80,         # Batas token untuk ringkasan
        "temperature": 0.3,     # Rendah agar ringkasan tetap faktual
        "stop_sequences": "<|endoftext|>" # Tanda berhenti untuk model
    }

    try:
        # 'output_iterator' adalah aliran (stream) dari token/kata-kata
        output_iterator = replicate.run(model_identifier, input=input_data)

        # Kita harus menggabungkan semua kata dari aliran tersebut
        full_response = "".join(output_iterator)

        return full_response.strip()

    except Exception as e:
        return f"[Error memanggil Replicate: {e}]"


Kredensial Replicate telah diatur.


In [17]:
import time

# Ambil 3 sampel ulasan acak dari dataset asli
sample_reviews = df.sample(3, random_state=42)

for index, row in sample_reviews.iterrows():
    original_review = row['Review Text']

    # 1. Jalankan Klasifikasi (Model dari Colab)
    cleaned_review_for_classification = clean_text(original_review)
    sentiment_prediction = classification_model.predict([cleaned_review_for_classification])[0]
    sentiment = "Positif" if sentiment_prediction == 1 else "Negatif"

    summary = summarize_review(original_review)
    print("="*70)
    print(f"ULASAN ASLI:\n{original_review}\n")
    print(f"   -> [HASIL KLASIFIKASI]: Sentimen diprediksi {sentiment}")
    print(f"   -> [HASIL PERINGKASAN]: {summary}")
    print("="*70, "\n")

    time.sleep(10)


ULASAN ASLI:
This sweater is so beautiful on. it is thick material, but does not make you look boxy. it fits so nicely and is flattering. the design is just gorgeous. if you are considering this sweater-- get it, you won't regret it!

i'm normally a small (4-6) and i ordered a small. it fits true to size. the fit is *perfect* so if you want it slightly more relaxed, order one size up.

   -> [HASIL KLASIFIKASI]: Sentimen diprediksi Positif
   -> [HASIL PERINGKASAN]: The sweater, fitting true to size for a small (4-6), offers a flattering, well-designed, and high-quality fit that is both thick and stylish, highly recommended for its beauty and comfort.

ULASAN ASLI:
This piece is almost what i want... i tried on the white version in an xs and it felt a little too large for me, even though it's meant to be a looser fit. the buttercup yellow behind the lace is very pretty - i especially love that part of this tank. it provides a nice contrast and allows the detail to stand out. the navy i